In [12]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [13]:

from scipy import stats
import numpy as np


df = pd.read_csv('SolarPredictionfinal.csv')

column_to_clean = 'Radiation'

z_scores = stats.zscore(df[column_to_clean])

threshold = 3

outlier_indices = (np.abs(z_scores) > threshold)

df_no_outliers = df[~outlier_indices]

df_no_outliers.reset_index(drop=True, inplace=True)

# Display the cleaned dataset
print(df_no_outliers)

threshold = df_no_outliers['Radiation'].median()
y_binary = np.where(df_no_outliers['Radiation'] > threshold, 1, 0)


         UNIXTime            Data      Time  Radiation  Temperature  Pressure  \
0      1475229326  9/29/2016 0:00  23:55:26       1.21           48     30.46   
1      1475229023  9/29/2016 0:00  23:50:23       1.21           48     30.46   
2      1475228726  9/29/2016 0:00  23:45:26       1.23           48     30.46   
3      1475228421  9/29/2016 0:00  23:40:21       1.21           48     30.46   
4      1475228124  9/29/2016 0:00  23:35:24       1.17           48     30.46   
...           ...             ...       ...        ...          ...       ...   
32605  1480587604  12/1/2016 0:00   0:20:04       1.22           44     30.43   
32606  1480587301  12/1/2016 0:00   0:15:01       1.17           44     30.42   
32607  1480587001  12/1/2016 0:00   0:10:01       1.20           44     30.42   
32608  1480586702  12/1/2016 0:00   0:05:02       1.23           44     30.42   
32609  1480586402  12/1/2016 0:00   0:00:02       1.20           44     30.43   

       Humidity  WindDirect

In [14]:
class_column = "Radiation" 
total_instances = len(df_no_outliers)
class_counts = df_no_outliers[class_column].value_counts()
prior_probabilities = class_counts / total_instances

# Print or use the prior probabilities as needed
print(prior_probabilities)


1.22      0.069427
1.23      0.064704
1.21      0.061638
1.24      0.052898
1.20      0.047777
            ...   
402.58    0.000031
383.56    0.000031
346.11    0.000031
301.83    0.000031
4.29      0.000031
Name: Radiation, Length: 14359, dtype: float64


In [16]:
import pandas as pd
from io import StringIO

data = """
age,income,student,credit_rating,buys_computer
<=30,high,no,fair,no
<=30,high,no,excellent,no
31...40,high,no,fair,yes
>40,medium,no,fair,yes
>40,low,yes,fair,yes
>40,low,yes,excellent,no
31...40,low,no,fair,yes
<=30,medium,no,excellent,yes
<=30,low,yes,fair,yes
>40,medium,yes,fair,yes
<=30,medium,yes,excellent,yes
31...40,medium,no,excellent,yes
31...40,high,yes,fair,yes
>40,medium,no,excellent,no
"""

df1 = pd.read_csv(StringIO(data))

class_column = 'buys_computer'


total_instances = len(df1)
class_counts = df1[class_column].value_counts()
prior_probabilities = class_counts / total_instances

# Print or use the prior probabilities as needed
print(prior_probabilities)


yes    0.714286
no     0.285714
Name: buys_computer, dtype: float64


In [23]:

from sklearn.naive_bayes import GaussianNB



class_column = 'Radiation'
numeric_columns = df_no_outliers.select_dtypes(include=['number']).columns
X = df_no_outliers[numeric_columns]

y = np.where(df_no_outliers['Radiation'] > threshold, 1, 0)


gnb = GaussianNB()


gnb.fit(X, y)
class_conditional_densities = {}
for class_label in gnb.classes_:
    class_conditional_densities[class_label] = {}
    for feature_idx, feature_name in enumerate(X.columns):
        class_conditional_densities[class_label][feature_name] = {
            'mean': gnb.theta_[class_label, feature_idx],
            'variance': gnb.var_[class_label, feature_idx]
        }

for class_label, features in class_conditional_densities.items():
    print(f"Class: {class_label}")
    for feature_name, values in features.items():
        print(f"Feature: {feature_name}")
        print(f"Mean: {values['mean']}, Variance: {values['variance']}")
        print()


has_zero_values = any(
    any(value == 0 for value in values.get('variance', [1])) 
    for values in class_conditional_densities.values()
)

print(f"Any class conditional density has zero values: {has_zero_values}")


Class: 0
Feature: UNIXTime
Mean: 1478278759.7541847, Variance: 8623283758244.693

Feature: Radiation
Mean: 1.3026954442332062, Variance: 9026.863530113054

Feature: Temperature
Mean: 47.40223189649887, Variance: 9039.292462963058

Feature: Pressure
Mean: 30.42275982586301, Variance: 9026.798692720939

Feature: Humidity
Mean: 75.96829971181556, Variance: 9716.894599954376

Feature: WindDirection(Degrees)
Mean: 158.56558035440548, Variance: 11578.726925803974

Feature: Speed
Mean: 6.236451039303036, Variance: 9038.21162283964

Class: 1
Feature: UNIXTime
Mean: 1477827657.0831852, Variance: 9328733716970.426

Feature: Radiation
Mean: 408.2100184037775, Variance: 121270.48160805595

Feature: Temperature
Mean: 54.76676277528986, Variance: 9063.890085699486

Feature: Pressure
Mean: 30.42287528372486, Variance: 9026.798486697206

Feature: Humidity
Mean: 74.01981473529231, Variance: 9687.513505598212

Feature: WindDirection(Degrees)
Mean: 128.60053370958892, Variance: 19826.409779944315

Featur

In [26]:
from scipy.stats import chi2_contingency
features_to_test = ['Temperature', 'Pressure', 'Humidity', 'Speed']


for i in range(len(features_to_test) - 1):
    for j in range(i + 1, len(features_to_test)):
        contingency_table = pd.crosstab(df_no_outliers[features_to_test[i]], df_no_outliers[features_to_test[j]])
        chi2, p, _, _ = chi2_contingency(contingency_table)
        
        print(f"Chi-squared test for independence between {features_to_test[i]} and {features_to_test[j]}:")
        print(f"Chi-squared statistic: {chi2}")
        print(f"P-value: {p}")
        
        # Check the significance level (commonly set to 0.05)
        alpha = 0.05
        print(f"Is the result significant at the {alpha} level? {'Yes' if p < alpha else 'No'}")
        print("\n")



Chi-squared test for independence between Temperature and Pressure:
Chi-squared statistic: 21126.995618982797
P-value: 0.0
Is the result significant at the 0.05 level? Yes


Chi-squared test for independence between Temperature and Humidity:
Chi-squared statistic: 29725.75354501293
P-value: 0.0
Is the result significant at the 0.05 level? Yes


Chi-squared test for independence between Temperature and Speed:
Chi-squared statistic: 2641.3738115624064
P-value: 7.712527115958194e-89
Is the result significant at the 0.05 level? Yes


Chi-squared test for independence between Pressure and Humidity:
Chi-squared statistic: 22729.635528772116
P-value: 0.0
Is the result significant at the 0.05 level? Yes


Chi-squared test for independence between Pressure and Speed:
Chi-squared statistic: 30132.625103238228
P-value: 0.0
Is the result significant at the 0.05 level? Yes


Chi-squared test for independence between Humidity and Speed:
Chi-squared statistic: 7712.730881017505
P-value: 0.0
Is the re

In [29]:

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X = df_no_outliers[['Temperature', 'Humidity', 'Speed', 'Pressure']]
y = df_no_outliers['Radiation']  
y = np.where(df_no_outliers['Radiation'] > threshold, 1, 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.77
Confusion Matrix:
[[2838  411]
 [1060 2213]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.87      0.79      3249
           1       0.84      0.68      0.75      3273

    accuracy                           0.77      6522
   macro avg       0.79      0.77      0.77      6522
weighted avg       0.79      0.77      0.77      6522



In [47]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder

X = df1.drop(columns=['buys_computer'])
y = df1['buys_computer']

# Perform one-hot encoding for categorical variables
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X)

# Initialize Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Fit the model to the data
gnb.fit(X_encoded, y)

# Class conditional densities for each feature and class
class_conditional_densities = {}
for class_label, class_idx in zip(gnb.classes_, range(len(gnb.classes_))):
    class_conditional_densities[class_label] = {}
    for feature_idx, feature_name in enumerate(encoder.get_feature_names_out()):
        class_conditional_densities[class_label][feature_name] = {
            'mean': gnb.theta_[class_idx, feature_idx],
            'variance': gnb.var_[class_idx, feature_idx]
        }

# Print or use the class conditional densities as needed
for class_label, features in class_conditional_densities.items():
    print(f"Class: {class_label}")
    for feature_name, values in features.items():
        print(f"Feature: {feature_name}")
        print(f"Mean: {values['mean']}, Variance: {values['variance']}")
        print()

has_zero_values = any(
    any(value == 0 for value in values.get('variance', [1])) 
    for values in class_conditional_densities.values()
)

print(f"Any class conditional density has zero values: {has_zero_values}")


Class: no
Feature: age_<=30
Mean: 0.5, Variance: 0.25000000024489794

Feature: age_>40
Mean: 0.5, Variance: 0.25000000024489794

Feature: income_low
Mean: 0.25, Variance: 0.18750000024489796

Feature: income_medium
Mean: 0.25, Variance: 0.18750000024489796

Feature: student_yes
Mean: 0.25, Variance: 0.18750000024489796

Feature: credit_rating_fair
Mean: 0.25, Variance: 0.18750000024489796

Class: yes
Feature: age_<=30
Mean: 0.3, Variance: 0.21000000024489793

Feature: age_>40
Mean: 0.3, Variance: 0.21000000024489798

Feature: income_low
Mean: 0.3, Variance: 0.21000000024489798

Feature: income_medium
Mean: 0.5, Variance: 0.25000000024489794

Feature: student_yes
Mean: 0.5, Variance: 0.25000000024489794

Feature: credit_rating_fair
Mean: 0.7, Variance: 0.21000000024489798

Any class conditional density has zero values: False


C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X = df1[['age', 'income', 'student', 'credit_rating']]
y = df1['buys_computer']  
X_encoded = encoder.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.33
Confusion Matrix:
[[0 1]
 [1 1]]
Classification Report:
              precision    recall  f1-score   support

          no       0.00      0.00      0.00         1
         yes       0.50      0.50      0.50         2

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3



C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

X = df1.drop(columns=['buys_computer'])
y = df1['buys_computer']

X_encoded = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)


Accuracy: 0.6666666666666666
Classification Report:
              precision    recall  f1-score   support

          no       0.00      0.00      0.00         1
         yes       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
